In [ ]:
from models import GAN, SL2Z_DataModule
from pytorch_lightning import Trainer

In [ ]:
model = GAN()
sl2z_dm = SL2Z_DataModule(max_length = 10, num_dataset = 100000, batch_size=2056)

In [ ]:
trainer = Trainer(
    max_epochs = 1000,
    accelerator = 'auto',
    logger = False
    )

In [ ]:
trainer.fit(model, sl2z_dm)

In [ ]:
samples = model.generate_samples(20)

In [ ]:
samples

In [ ]:
import matrix_methods as mm
mm.det_batch(samples.detach().numpy())

In [ ]:
mm.det_batch(sl2z_dm.matrices.numpy())